# BUSSINESS AND DATA UNDERSTANDING

Este notebook esta destinado a cubrir las dos primeras fases estipulada en la metodología  CRISP-DM (Cross Industry Standard Process for Data Mining).

La metodología se divide en las siguientes: 

1. <span style="color:green;">[**Comprensión del Negocio (Business Understanding)**](#business-understanding)</span>  
   - Consistente en el entendimiento del objetivo del proyecto.

2. <span style="color:green;">[**Comprensión de los Datos (Data Understanding)**](#data-understanding)</span>  
   - Relacionada con la carga y primera evaluación del conjunto de datos.

3. **Preparación de los Datos (Data Preparation)**  
   - Consistente en la limpieza, preparación y extracción de características de los datos.

4. **Modelado (Modeling)**  
   - Relacionada con la elección del modelo de machine learning y el ajuste hiperparamétrico.

5. **Evaluación (Evaluation)**  
   - Evaluación de los resultados obtenidos por el modelo.

6. **Implementación (Deployment)**  
   - Integración del modelo de forma que sea accesible para su uso.





<a id="business-understanding"></a>

## 1. Business understanding

El problema a tratar se trata de la obtención de descripciones de las escenas a partir de los cotidianos presentes en las mismas, con enfasis en aquellos elementos que sean especialmente relevantes para la mejora de la accesibilidad en personas con discapacidad visual, 


Es por ello que podemos definir este problema como uno de **segmentación semántica**, al ser necesaria la obtención de regiones dentro de una imagen de entrada que se correspondan con los objetos a detectar, de tal forma que cada uno de los píxeles de esta quede clasificado como una clase de un conjunto de clases finito, diferenciandose así de  otros problemas como el de clasificación de imágenes, donde se asigna una etiqueta o categoría a la imagen de forma global, el de detección, donde se localizan objetos delimitandolos por cajas o bounding boxes, o el de segmentación de instancias, que identifica cada instancia de cada objeto de forma separada. Un ejemplo práctico cada uno de los problemas anteriores se observa en la figura siguiente:


<img src="./../assets/figs/problema.png" alt="Problema a resolver" style="width: 50%; height: auto;">



<a id="data-understanding"></a>

## 2. Data understanding

Una vez definido el problema, se procede a la carga del conjunto de datos.

Esta etapa se divide en hasta 4 fases:

1. Descarga del conjunto de datos
2. Análisis exploratorio del conjunto de datos
3. Sampling del conjunto de datos
4. Extracción de características


Para una mejor organización, en este notebook, se abordará la primera de las 4 fases


En este caso se ha decidido hacer uso del dataset `MS COCO` en su más reciente actualización, que data del 2017, siendo posible acceder a su documentación en el siguiente [enlace](https://cocodataset.org/#home), frente a otros datasets como `ADE20K`, en parte gracias a su alto volumen de datos y mayor variedad de clases etiquetadas, por ser más facil de moldar al caso de uso definido.

Este dataset destaca por contener, en su versión mas reciente, hasta un total de casi 200.000 imágenes, en las cuales se encuentran hasta un total de 1.5 millones de objetos diferentes segmentados de hasta un total de 80 clases diferetes. Fue propuesto por Microsoft en 2014 y actualizado en 2017 y puede ser utilizado para resolver problemas de detección de objetos, segmentación semántica y segmentación de instancias, además de contener keypoints de las 17 partes del cuerpo humano más relevante para la detección de poses. Cabe destacar una gran variedad de resoluciones de las imágenes, fuentes de datos de su obtención, al ser obtenidas a partir de plataformas como Flickr y de una variedad de geografías, que hacen que ayudan a que el entrenamiento de un modelo con este conjunto de datos pueda generalizar mejor.





Existen dos formas de cargarlo en memoria, por un lado puede ser cargado a disco directamente descargando los datos de su web, o bien a partir de la libreria `tensorflow_datasets`, que dejará el dataset almacenado en memorai de forma temporal.


Podemos almacenarlo de forma permanente de la siguiente manera: 

In [36]:
import importlib
import utils

# refresca la cache para la carga adecuada del fichero utils, donde no siempre captura bien las funciones
importlib.reload(utils)


<module 'utils' from 'c:\\Users\\ruben\\Desktop\\code_tfm\\src\\utils.py'>

In [41]:
from pathlib import Path
from utils import  load_yaml_file, download_zip
import os

Se cargan las constantes necesarias definidas en el YALM

In [50]:
yaml = load_yaml_file()

# Constantes relativas a la direccion destino de la carga de imagenes del dataset

DIR_IMGS_DATASET_RAW = yaml["dir_datasets"]

In [51]:

# Nos colocamos en la raiz para que el path sea el adecuado

DIR_IMGS_DATASET_RAW 
dir_data_load = os.path.join(os.getcwd(),"..",DIR_IMGS_DATASET_RAW)
dir_data_load = os.path.normpath(dir_data_load)
dir_data_load

'c:\\Users\\ruben\\Desktop\\code_tfm\\dataset\\coco'

Se procede a la descarga automatizada de estos en los directorios destino, junto con sus archivos de anotación correspondiente:

In [55]:

segments = True  # True: se incluye la segmentacion, al tratar de resolverse un problema de segmentación y no de detección 

dir = Path(DIR_IMGS_DATASET_RAW)  

URL_BASE = 'https://github.com/ultralytics/assets/releases/download/v0.0.0/'

# Fichero usado para obtener las labels
urls = [URL_BASE + ('coco2017labels-segments.zip' if segments else 'coco2017labels.zip')]
# Imágenes
urls += [
    'http://images.cocodataset.org/zips/train2017.zip',
    'http://images.cocodataset.org/zips/val2017.zip',
    'http://images.cocodataset.org/zips/test2017.zip',
]

destinos = [DIR_IMGS_DATASET_RAW + nombre_destino for  nombre_destino in ["segments", "train", "val", "test"] ]
filenames = ["segments.zip","train2017.zip", "val2017.zip", "test2017.zip"]



In [57]:
urls

['https://github.com/ultralytics/assets/releases/download/v0.0.0/coco2017labels-segments.zip',
 'http://images.cocodataset.org/zips/train2017.zip',
 'http://images.cocodataset.org/zips/val2017.zip',
 'http://images.cocodataset.org/zips/test2017.zip']

Se procede a la descarga

In [ ]:
for url, destino, nombre in zip(urls, destinos, filenames):
    print("iterating over",url, destino, nombre)
    download_zip(url, destino, nombre)



Se carga alguna imagen para comprobar la carga correcta del conjunto de datos

In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors
import seaborn as sns
import numpy as np

from random import shuffle
from PIL import Image

from pycocotools.coco import COCO

In [ ]:
dataDir='../datasets/coco/'
dataType='val2017'
annFile='{}annotations/instances_{}.json'.format(dataDir,dataType)
imageDir = '{}/images/{}/'.format(dataDir, dataType)

# Initialize the COCO api for instance annotations
coco=COCO(annFile)

In [ ]:
# Load categories for the given ids
ids = 1
cats = coco.loadCats(ids=ids)
print(cats)

In [ ]:
category_ids = coco.getCatIds()
num_categories = len(category_ids)
print('number of categories: ',num_categories)
for ids in category_ids:
    cats = coco.loadCats(ids=ids)
    print(cats)

In [ ]:
# Load images for the given ids
image_ids = coco.getImgIds()
image_id = image_ids[0]  # Change this line to display a different image
image_info = coco.loadImgs(image_id)
print(image_info)

In [ ]:
# Load annotations for the given ids
annotation_ids = coco.getAnnIds(imgIds=image_id)
annotations = coco.loadAnns(annotation_ids)
print(annotations)

In [ ]:
# Get category ids that satisfy the given filter conditions
filterClasses = ['laptop', 'tv', 'cell phone']
# Fetch class IDs only corresponding to the filterClasses
catIds = coco.getCatIds(catNms=filterClasses)
print(catIds)

In [ ]:
# Load category information for the given ID
catID = 15
print(coco.loadCats(ids=catID))

# Get image ID that satisfies the given filter conditions
imgId = coco.getImgIds(catIds=[catID])[0]
print(imgId)

In [ ]:
catID = 15
print(coco.loadCats(ids=catID))

# Get image ids that satisfy the given filter conditions
imgId = coco.getImgIds(catIds=[catID])[0]
print(imgId)

In [ ]:
ann_ids = coco.getAnnIds(imgIds=[imgId], iscrowd=None)
print(ann_ids)

In [ ]:
print(f"Annotations for Image ID {imgId}:")
anns = coco.loadAnns(ann_ids)

image_path = coco.loadImgs(imgId)[0]['file_name']
print(image_path)
image = plt.imread(imageDir + image_path)
plt.imshow(image)

# Display the specified annotations
coco.showAnns(anns, draw_bbox=True)

plt.axis('off')
plt.title('Annotations for Image ID: {}'.format(image_id))
plt.tight_layout()
plt.show()

Se comprueba tambien que la carga de los segmentos ha sido la adecuada